In [21]:
source = "ZRH"
destination = "MXP"
startdate = "2019-01-01"
enddate = "2019-12-01"

url = "https://www.kayak.com/flights/" + origin + "-" + destination + "/" + startdate + "/"

In [22]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome("chromedriver")
driver.implicitly_wait(20)
driver.get(url)
time.sleep(3)

In [23]:
soup=BeautifulSoup(driver.page_source, 'lxml')

deptimes = soup.find_all('span', attrs={'class': 'depart-time base-time'})
arrtimes = soup.find_all('span', attrs={'class': 'arrival-time base-time'})
meridies = soup.find_all('span', attrs={'class': 'time-meridiem meridiem'})
    
deptime = []
for div in deptimes:
    deptime.append(div.getText()[:-1])    
        
arrtime = []
for div in arrtimes:
    arrtime.append(div.getText()[:-1])   
 
meridiem = []
for div in meridies:
    meridiem.append(div.getText())  
        
deptime = np.asarray(deptime)
deptime = deptime.reshape(int(len(deptime)/2), 2)
    
arrtime = np.asarray(arrtime)
arrtime = arrtime.reshape(int(len(arrtime)/2), 2)      
    
meridiem = np.asarray(meridiem)
meridiem = meridiem.reshape(int(len(meridiem)/4), 4)

In [24]:
regex = re.compile('Common-Booking-MultiBookProvider (.*)multi-row Theme-featured-large(.*)')
price_list = soup.find_all('div', attrs={'class': regex})
    
price = []
for div in price_list:
    price.append(int(div.getText().split('\n')[3][1:-1]))

In [27]:
df = pd.DataFrame({"Airlines": airlines,
                   "Date of journey": date_of_journey,
                   "Source" : source,
                   "destination" : destination,
                   "Route" : route,
                   "Dep_Time": [m+str(n) for m,n in zip(deptime[:,0],meridiem[:,0])],
                   "Arrival_Time": [m+str(n) for m,n in zip(arrtime[:,0],meridiem[:,1])],
                   "Duration":duration,
                   "Total_Stops":total_stops,
                   "Additional_Info":add_info,
                   "Price":price
                   })

In [31]:
df.to_excel("Data_Train.xlsx")
print("File saved")

File saved


In [34]:
df1=pd.read_excel("Data_train.xlsx")
df1.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [35]:
df1.tail()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648
10682,Air India,9/05/2019,Delhi,Cochin,DEL → GOI → BOM → COK,10:55,19:15,8h 20m,2 stops,No info,11753
